In [86]:
# This script finds the path to the 'data' folder in the parent directory of the current notebook.
import os

# Get the directory of the current notebook
notebook_directory = os.getcwd()
print(f"Current notebook directory: {notebook_directory}")

# Go up one level to the main project directory
parent_directory = os.path.dirname(notebook_directory)
print(f"Parent directory: {parent_directory}")

# Specify the path to the 'data' folder from the parent directory
data_folder_path = os.path.join(parent_directory, 'data')
print(f"Data directory: {data_folder_path}")

Current notebook directory: c:\08_AHFID\gbv-predictive-tool\notebooks
Parent directory: c:\08_AHFID\gbv-predictive-tool
Data directory: c:\08_AHFID\gbv-predictive-tool\data


In [87]:
import pandas as pd
file_path = os.path.join(data_folder_path, 'NGBV Dashboard dataset.xlsx')
df = pd.read_excel(file_path, header=2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44360 entries, 0 to 44359
Data columns (total 78 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Time Stamp Date                                              44350 non-null  object 
 1   Time Stamp Time                                              44350 non-null  object 
 2   Type_of_Organisation                                         44350 non-null  object 
 3   Location of Organisation State                               44350 non-null  object 
 4   Organisation LGA                                             38944 non-null  object 
 5   Contact Channel                                              41821 non-null  object 
 6   Contact Channel Other                                        2515 non-null   object 
 7   Was the Violence Fatal                                       44350 non-null 

In [105]:
pd.set_option('display.max_columns', None)  # Show all columns in the DataFrame

In [106]:
print("The original shape of the DataFrame:", df.shape)
print("The original columns of the DataFrame:", df.columns.tolist())

The original shape of the DataFrame: (44360, 32)
The original columns of the DataFrame: ['Type_of_Organisation', 'Was the Violence Fatal', 'Sex of survivor', 'Age of survivor', 'MARITAL STATUS', 'DOES THE SURVIVOR/VICTIM LIVE ALONE', 'ESTIMATED AVERAGE MONTHLY INCOME', 'DATE OF INCIDENT', 'DATE REPORTED', 'LOCATION OF VIOLENCE (STATE)', 'LOCATION OF VIOLENCE (L.G.A)', 'LOCATION OF VIOLENCE (WARD)', 'TIME OF THE DAY THAT INCIDENT TOOK PLACE', 'DOES THE SURVIVOR WANT ACCESS TO JUSTICE', 'OUTCOME OF PROSECUTION', 'DATE JUSTICE WAS RECEIVED', 'HAS THE CASE BEEN CLOSED', 'WHO CLOSED THE CASE?', 'DATE CASE WAS CLOSED', 'APPROVED BY ORG. SUPERVISOR_DATE', 'APPROVED BY LGA SUPERVISOR_DATE', 'APPROVED BY STATE SUPERVISOR_DATE', 'Location of Organisation', 'WHO REPORTED THE INCIDENT', 'PARENT/GUARDIAN EMPLOYMENT STATUS', 'SURVIVOR/VICTIM EMPLOYMENT STATUS', 'WHO SURVIVOR/VICTIM LIVE WITH', 'SEX OF PERPETRATOR', 'AGE OF PERPETRATOR', 'RELATIONSHIP WITH PERPETRATOR', 'VULNERABLE POPULATION', 'TYPE

### **Merge several related columns into a single, consolidated column:**

**General Strategy:**
*   Identify Groups: Group the columns that need to be merged.
*   Define New Column Name: Decide on the name for the consolidated column.
*   Combine Values: For each row, iterate through the columns in a group
*   Create New Column: Assign the combined values to the new consolidated column.
*   Drop Old Columns: Remove the original, now redundant, columns.

In [107]:
def combine_columns(df, new_col_name, cols_to_combine, separator=','):
    """
    Combine multiple columns into a new column with a separator, ignoring NaN.

    Args:
        df: The DataFrame to modify
        new_col_name: The new name of the combined column
        cols_to_combine (list): The list of the column names to combine
        separator: The string to use to join the multiple non-NaN values

    Returns:
        The modified DataFrame with the new column and old ones dropped
    """
    missing_cols = [col for col in cols_to_combine if col not in df.columns]
    if missing_cols:
        raise ValueError(f"The following columns do not exist in the DataFrame: {missing_cols}")

    cols_to_combine = [col for col in cols_to_combine if col in df.columns]
    if not cols_to_combine:
        raise ValueError("No valid columns to combine")

    df[new_col_name] = df[cols_to_combine].apply(
        lambda row: separator.join(row.dropna().astype(str)),
        axis=1
    )

    df[new_col_name] = df[new_col_name].replace('', pd.NA)

    df = df.drop(columns=cols_to_combine)

    return df

In [108]:
# 1. Location of Organisation
df = combine_columns(df, 'Location of Organisation', ['Location of Organisation State', 'Organisation LGA'])

# 2. Who reported the incident
df = combine_columns(df, 'WHO REPORTED THE INCIDENT', ['Who Reported the Incident', 'WHO REPORTED THE INCIDENT_OTHER'])

# 3. Contact Channel
df = combine_columns(df, 'Contact Channel', ['Contact Channel', 'Contact Channel Other'])

# 4. Parent/Guardian Employment Status
df = combine_columns(df, 'PARENT/GUARDIAN EMPLOYMENT STATUS', ['EMPLOYMENT STATUS OF PARENT/GUARDIAN', 'EMPLOYMENT STATUS OF PARENT/GUARDIAN_OTHERS'])

# 5. Survivor/Victim Employment Status
df = combine_columns(df, 'SURVIVOR/VICTIM EMPLOYMENT STATUS', ['EMPLOYMENT STATUS OF SURVIVOR/VICTIM', 'EMPLOYMENT STATUS OF SURVIVOR/VICTIM_OTHERS'])

# 6. Educational Status
df = combine_columns(df, 'EDUCATIONAL STATUS', ['EDUCATIONAL STATUS', 'EDUCATIONAL STATUS_OTHER'])

# 7. Who does the survivor/victim live with
df = combine_columns(df, 'WHO SURVIVOR/VICTIM LIVE WITH', ['WHO DOES THE SURVIVOR/VICTIM LIVE WITH', 'WHO DOES THE SURVIVOR/VICTIM LIVE WITH_OTHER'])

# 8. Perpetrator's Sex
df = combine_columns(df, 'SEX OF PERPETRATOR', ['SEX OF PERPETRATOR_1', 'SEX OF PERPETRATOR_2', 'SEX OF PERPETRATOR_3', 'SEX OF PERPETRATOR_OTHER'])

# 9. Perpetrator's Age
df = combine_columns(df, 'AGE OF PERPETRATOR', ['AGE OF PERPETRATOR_1', 'AGE OF PERPETRATOR_2', 'AGE OF PERPETRATOR_3', 'AGE OF PERPETRATOR_OTHER'])

# 10 Survivor/Victim's Relationship with Perpetrator
df = combine_columns(
    df,
    "RELATIONSHIP WITH PERPETRATOR",
    [
        "SURVIVOR/VICTIM'S RELATIONSHIP WITH PERPETRATOR_1",
        "SURVIVOR/VICTIM'S RELATIONSHIP WITH PERPETRATOR_2",
        "SURVIVOR/VICTIM'S RELATIONSHIP WITH PERPETRATOR_3",
        "SURVIVOR/VICTIM'S RELATIONSHIP WITH PERPETRATOR_OTHER"
    ]
)

# # 11. Location of Violence
# df = combine_columns(df, 'LOCATION OF VIOLENCE', ['LOCATION OF VIOLENCE (STATE)', 'LOCATION OF VIOLENCE (L.G.A)', 'LOCATION OF VIOLENCE (WARD)'])

# 12. Place of Incident
df = combine_columns(df, 'PLACE OF INCIDENT', ['PLACE OF INCIDENT', 'PLACE OF INCIDENT_OTHER'])


print("\nProcessed DataFrame shape:", df.shape)
print("\nNew DataFrame Columns:")
print(df.columns.tolist())

ValueError: The following columns do not exist in the DataFrame: ['Location of Organisation State', 'Organisation LGA']

### **Collapse multiple binary columns into a single column "VULNERABLE POPULATION" and "TYPE OF VIOLENCE"**
**What should be done:**
* Loop through the binary columns.
* For each row, collect labels of columns marked “Yes”.
* Store them as a comma-separated list in a new column.


In [91]:
#@title Collapse multiple "VULNERABLE POPULATION_" columns into a single "VULNERABLE POPULATION"
def combine_vulnerable_population_columns(df):
    # Step 1: Select relevant columns
    vuln_cols = [col for col in df.columns if col.startswith("VULNERABLE POPULATION_")]

    # Step 2: Strip the prefix to get clean category names
    category_map = {col: col.replace("VULNERABLE POPULATION_", "").strip() for col in vuln_cols}

    # Step 3: Apply row-wise logic
    def get_vuln_categories(row):
        categories = [category_map[col] for col in vuln_cols if str(row[col]).strip().upper() == "YES"]
        return ", ".join(categories) if categories else pd.NA

    # Step 4: Create the new column
    df["VULNERABLE POPULATION"] = df.apply(get_vuln_categories, axis=1)

    # Optional: Drop the original binary columns
    df = df.drop(columns=vuln_cols)

    return df

df = combine_vulnerable_population_columns(df)

print("\nProcessed DataFrame shape:", df.shape)
print("\nNew DataFrame Columns:")
print(df.columns.tolist())


Processed DataFrame shape: (44360, 46)

New DataFrame Columns:
['Time Stamp Date', 'Time Stamp Time', 'Type_of_Organisation', 'Was the Violence Fatal', 'Sex of survivor', 'Age of survivor', 'MARITAL STATUS', 'DOES THE SURVIVOR/VICTIM LIVE ALONE', 'ESTIMATED AVERAGE MONTHLY INCOME', 'DATE OF INCIDENT', 'DATE REPORTED', 'LOCATION OF VIOLENCE (STATE)', 'LOCATION OF VIOLENCE (L.G.A)', 'LOCATION OF VIOLENCE (WARD)', 'TIME OF THE DAY THAT INCIDENT TOOK PLACE', 'TYPE OF VIOLENCE_SEXUAL ASSAULT', 'TYPE OF VIOLENCE_PHYSICAL ASSAULT', 'TYPE OF VIOLENCE_FINANCIAL/ECONOMIC', 'TYPE OF VIOLENCE_ONLINE/CYBER', 'TYPE OF VIOLENCE_RAPE', 'TYPE OF VIOLENCE_DEFILEMENT', 'TYPE OF VIOLENCE_FORCED MARRIAGE', 'TYPE OF VIOLENCE_DENIAL OF RESOURCES', 'TYPE OF VIOLENCE_PSYCHOLOGICAL/EMOTIONAL ABUSE', 'TYPE OF VIOLENCE_FEMALE GENITAL MUTILATION', 'TYPE OF VIOLENCE_VIOLATION OF PROPERTY & INHERITANCE RIGHTS', 'TYPE OF VIOLENCE_CHILD ABUSE AND NEGLECT', 'TYPE OF VIOLENCE_OTHER', 'DOES THE SURVIVOR WANT ACCESS TO J

In [92]:
#@title Collapse multiple "TYPE OF VIOLENCE_" columns into a single "TYPE OF VIOLENCE"
def combine_violence_types(df):
    # Step 1: Get all relevant columns
    violence_cols = [col for col in df.columns if col.startswith("TYPE OF VIOLENCE_")]

    # Step 2: Map column names to clean labels
    type_map = {col: col.replace("TYPE OF VIOLENCE_", "").strip() for col in violence_cols}

    # Step 3: Apply row-wise logic
    def get_violence_types(row):
        types = [type_map[col] for col in violence_cols if str(row[col]).strip().upper() == "YES"]
        return ", ".join(types) if types else pd.NA

    # Step 4: Add new column
    df["TYPE OF VIOLENCE"] = df.apply(get_violence_types, axis=1)

    # Step 5: Drop old flag columns
    df = df.drop(columns=violence_cols)

    return df

df = combine_violence_types(df)

print("\nProcessed DataFrame shape:", df.shape)
print("\nNew DataFrame Columns:")
print(df.columns.tolist())


Processed DataFrame shape: (44360, 34)

New DataFrame Columns:
['Time Stamp Date', 'Time Stamp Time', 'Type_of_Organisation', 'Was the Violence Fatal', 'Sex of survivor', 'Age of survivor', 'MARITAL STATUS', 'DOES THE SURVIVOR/VICTIM LIVE ALONE', 'ESTIMATED AVERAGE MONTHLY INCOME', 'DATE OF INCIDENT', 'DATE REPORTED', 'LOCATION OF VIOLENCE (STATE)', 'LOCATION OF VIOLENCE (L.G.A)', 'LOCATION OF VIOLENCE (WARD)', 'TIME OF THE DAY THAT INCIDENT TOOK PLACE', 'DOES THE SURVIVOR WANT ACCESS TO JUSTICE', 'OUTCOME OF PROSECUTION', 'DATE JUSTICE WAS RECEIVED', 'HAS THE CASE BEEN CLOSED', 'WHO CLOSED THE CASE?', 'DATE CASE WAS CLOSED', 'APPROVED BY ORG. SUPERVISOR_DATE', 'APPROVED BY LGA SUPERVISOR_DATE', 'APPROVED BY STATE SUPERVISOR_DATE', 'Location of Organisation', 'WHO REPORTED THE INCIDENT', 'PARENT/GUARDIAN EMPLOYMENT STATUS', 'SURVIVOR/VICTIM EMPLOYMENT STATUS', 'WHO SURVIVOR/VICTIM LIVE WITH', 'SEX OF PERPETRATOR', 'AGE OF PERPETRATOR', 'RELATIONSHIP WITH PERPETRATOR', 'VULNERABLE POPU

### **Drop 'Time Stamp Date' and 'Time Stamp Time'**

* 'Time Stamp Date' and 'Time Stamp Time' were the date and time the data were inputed. It is irrelevant to the incident data

In [93]:
#@title Drop 'Time Stamp Date' and 'Time Stamp Time'
df = df.drop(columns=['Time Stamp Date', 'Time Stamp Time'])
df.columns.tolist()

['Type_of_Organisation',
 'Was the Violence Fatal',
 'Sex of survivor',
 'Age of survivor',
 'MARITAL STATUS',
 'DOES THE SURVIVOR/VICTIM LIVE ALONE',
 'ESTIMATED AVERAGE MONTHLY INCOME',
 'DATE OF INCIDENT',
 'DATE REPORTED',
 'LOCATION OF VIOLENCE (STATE)',
 'LOCATION OF VIOLENCE (L.G.A)',
 'LOCATION OF VIOLENCE (WARD)',
 'TIME OF THE DAY THAT INCIDENT TOOK PLACE',
 'DOES THE SURVIVOR WANT ACCESS TO JUSTICE',
 'OUTCOME OF PROSECUTION',
 'DATE JUSTICE WAS RECEIVED',
 'HAS THE CASE BEEN CLOSED',
 'WHO CLOSED THE CASE?',
 'DATE CASE WAS CLOSED',
 'APPROVED BY ORG. SUPERVISOR_DATE',
 'APPROVED BY LGA SUPERVISOR_DATE',
 'APPROVED BY STATE SUPERVISOR_DATE',
 'Location of Organisation',
 'WHO REPORTED THE INCIDENT',
 'PARENT/GUARDIAN EMPLOYMENT STATUS',
 'SURVIVOR/VICTIM EMPLOYMENT STATUS',
 'WHO SURVIVOR/VICTIM LIVE WITH',
 'SEX OF PERPETRATOR',
 'AGE OF PERPETRATOR',
 'RELATIONSHIP WITH PERPETRATOR',
 'VULNERABLE POPULATION',
 'TYPE OF VIOLENCE']

In [109]:
df['TYPE OF VIOLENCE'].value_counts(dropna=False)[20:40]

TYPE OF VIOLENCE
SEXUAL ASSAULT, RAPE                                                                                                223
DENIAL OF RESOURCES, CHILD ABUSE AND NEGLECT                                                                        202
PHYSICAL ASSAULT, PSYCHOLOGICAL/EMOTIONAL ABUSE, CHILD ABUSE AND NEGLECT                                            185
PHYSICAL ASSAULT, CHILD ABUSE AND NEGLECT                                                                           167
DENIAL OF RESOURCES, PSYCHOLOGICAL/EMOTIONAL ABUSE, CHILD ABUSE AND NEGLECT                                         159
SEXUAL ASSAULT, PHYSICAL ASSAULT                                                                                    153
SEXUAL ASSAULT, PHYSICAL ASSAULT, PSYCHOLOGICAL/EMOTIONAL ABUSE                                                     149
SEXUAL ASSAULT, PSYCHOLOGICAL/EMOTIONAL ABUSE                                                                       130
PHYSICAL ASSAULT, DEFIL